<a href="https://colab.research.google.com/github/blusewill/ytvideo-whisper/blob/master/ytvideo_whisper_KMN_BOT_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ytvideo-whisper 機器狼特別版

此專案使用了 [Faster-Whisper](https://github.com/SYSTRAN/faster-whisper) 專案，而非原版的 Whisper 專案  

想使用原版 Whisper 專案的朋友，請查看原版 Whisper 分支  

[點此進入原版 Whisper 的 Colab](https://colab.research.google.com/github/blusewill/ytvideo-whisper/blob/original-whisper/ytvideo_whisper_KMN_BOT_Version.ipynb)  


# 使用方法

1. 請確認你的執行階段有啟用GPU，在 ``執行階段 -> 變更執行階段類型 ``
1. 更改設定
1. 按下 ``執行階段 -> 執行全部`` 或是 (CTRL+F9)
1. 然後連接到Google雲端硬碟
1. 然後等一下你的檔案應該就會在 ``雲端硬碟 -> Whisper -> result``

> 如果你的 Google Colab 當機的話，**請執行設定過後在按下生產字幕檔**

## 獲得Google雲端硬碟權限
(在 雲端硬碟 -> Whisper)

In [ ]:
#@markdown # **下載選項**
#@markdown ---
#@markdown ## **停用Google雲端硬碟**
disable_google_drive = "no" #@param ['yes', 'no']
# @markdown **當啟用停用 Google 雲端硬碟的時候 ``結束自動刪除執行階段`` 會直接停用。請記得使用後關機執行階段**
if disable_google_drive.lower() == 'no':

  from google.colab import drive
  import os

  path = '/content/gdrive/MyDrive/Whisper/'
  path2 = '/content/gdrive/MyDrive/Whisper/result'
  drive.mount('/content/gdrive')

  if not os.path.exists(path):
    os.mkdir(path)
    os.mkdir(path2)

# Calcuate the Execution Time.
from datetime import datetime
start_time = datetime.now()

Mounted at /content/gdrive


# 安裝程式
faster-whisper ctranslate2==4.5.0 yt-dlp ipython pysubs2

In [ ]:
! apt-get install ffmpeg
! pip install faster-whisper ctranslate2==4.5.0 yt-dlp ipython pysubs2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.2/37.2 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00


# 測試GPU是否有開啟

如果沒有東西跑出來的話，請去 ``執行階段 -> 變更執行階段類型`` 更改

In [ ]:
! nvidia-smi

Sun Dec  1 16:42:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 設定


In [ ]:
import os
#@markdown # **設定 (星號代表基礎設定)**
#@markdown ---
#@markdown ## **上傳自己的音檔 (目前只限 mp3 檔)**
upload_own_audio_file = "no" #@param ['yes', 'no']
#@markdown ## **YouTube 模式**
#@markdown ---
#@markdown ## ***影片連接**
# YouTube Link
YouTube_Video_Link = "https://www.youtube.com/watch?v=Y-hOsUinkm8" #@param {type:"string"}
# File Name Change
#@markdown ## ***檔案名稱**
new_filename = "【ChatGPT】自製GPT真的超簡單！機器狼的AI對話魔法！【GPTs】【入門筆記／精華版】#1-1" #@param {type:"string"}
#@markdown ## ***語言 (自動，中文，英文，日文)**
language = "Auto" #@param ["Auto", "zh", "en", "ja"]
#@markdown ## ***模式 ``生成 / 翻譯``**
task = "transcribe" #@param ['transcribe', 'translate']
# Change it to srt file type
new_filename = os.path.splitext(new_filename)[0] + ".srt"
new_filename2 = os.path.splitext(new_filename)[0] + " Transcript.txt"
# Change the Model of the Whisper
#@markdown # **其他選項**
#@markdown ---
#@markdown ## **生產逐字稿**
Generate_Plain_Document = "yes" #@param ['yes','no']
#@markdown ## **結束自動刪除執行階段**
shutdown_after_complete = "yes" #@param ['yes','no']
#@markdown ## **模型**
model_user = "medium" #@param ['tiny.en','tiny','base.en','base','small.en','small','medium.en','medium','large','large-v1','large-v2','large-v3','Custom']
#@markdown ## **自訂模型（請先在上面選擇 Custom）**
model_custom = "XA9/faster-whisper-large-v2-cantonese-1" #@param {type:"string"}
if model_user == "Custom":
  model_user = model_custom#@markdown ## **Cookies驗證**
enable_cookies = "no" #@param ['yes', 'no']

try:
  disable_google_drive
except NameError:
    print("Checking if the Google Drive is mounted")
    gdrive_crash_check = os.path.exists("/content/gdrive")
    if gdrive_crash_check == True:
      print("Crash Fallback : Google Drive is mounted!")
      disable_google_drive = "no"
    else:
      print("Crash Fallback : Google Drive is not mounted!")
      disable_google_drive = "yes"

# 如果你有啟動Cookies驗證
請在這裡上傳Cookies檔案

In [ ]:
import os
if enable_cookies.lower() == "yes":
  print("It is not recommended to upload your main account's cookies files here, as this is a shared environment.")
  print("Your cookies might get leaked, and I will not take any responsibility if your account gets stolen.")
  print("The recommended approach is to use another Google Account's cookies to run this tool.")
  print("If it is a private video, you can share the video with that Google Account to grant access to it.")
  print("")

  from google.colab import files
  cookies = files.upload()

  # Specify the path where the files are located
  path_cookies = "/content"

  def rename_files(path_cookies):
      for filename in os.listdir(path_cookies):
          file_path = os.path.join(path_cookies, filename)
          if os.path.isfile(file_path) and filename.endswith(".txt"):
              new_path = os.path.join(path_cookies, "cookies.txt")
              os.rename(file_path, new_path)

  rename_files(path_cookies)

# 下載影片/上傳音檔
使用yt-dlp跟ffmpeg

In [ ]:
from google.colab import files
import shutil
import yt_dlp

if upload_own_audio_file == "yes":
  if not os.path.isfile('audio.mp3'):
      own_audio = files.upload()
      if not os.path.exists('content'):
        os.mkdir('content')
        for filename in own_audio.keys():
            os.rename(filename, 'audio.mp3')
            os.replace('audio.mp3', 'content/audio.mp3')

else:
    if enable_cookies.lower() == "yes":
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'outtmpl': 'content/audio',
            'cookiefile': 'cookies.txt'
        }
    else:
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'outtmpl': 'content/audio',
        }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([YouTube_Video_Link])

[youtube] Extracting URL: https://www.youtube.com/watch?v=Y-hOsUinkm8
[youtube] Y-hOsUinkm8: Downloading webpage
[youtube] Y-hOsUinkm8: Downloading ios player API JSON
[youtube] Y-hOsUinkm8: Downloading mweb player API JSON
[youtube] Y-hOsUinkm8: Downloading player b46bb280
[youtube] Y-hOsUinkm8: Downloading m3u8 information
[info] Y-hOsUinkm8: Downloading 1 format(s): 251
[download] Destination: content/audio
[download] 100% of    3.24MiB in 00:00:00 at 7.50MiB/s   
[ExtractAudio] Destination: content/audio.mp3
Deleting original file content/audio (pass -k to keep)


# 生成字幕檔

In [ ]:
from faster_whisper import WhisperModel
import pysubs2
import os

try:
    start_time
except NameError:
    print("Crash Fallback : Starting to Calculate the Time from now")
    from datetime import datetime
    start_time = datetime.now()

# Load model
model = WhisperModel(model_user)
print("Whisper model loaded.")

filename = "audio.mp3"
input_directory = "content"
input_file = f"{input_directory}/{filename}"

# Transcribe with or without specific language setting
if task == "translate":
    segments, info = model.transcribe(input_file, task="translate", language=language)
else:
  if language.strip() == 'Auto':
      segments, info = model.transcribe(input_file)
      print("Detected language '%s' with probability %f" % (info.language, info.language_probability))
  else:
      segments, info = model.transcribe(input_file, language=language)

# Store results and show detailed information
results = []
print("Starting transcription with verbose output...")
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
    segment_dict = {'start': segment.start, 'end': segment.end, 'text': segment.text}
    results.append(segment_dict)

# Save to Google Drive or local directory as specified
if disable_google_drive.lower() == 'no':
    transcription_root = "/content/gdrive/MyDrive/Whisper/result/"
    os.makedirs(transcription_root, exist_ok=True)
else:
    transcription_root = "./result/"
    os.makedirs(transcription_root, exist_ok=True)

# Save as SRT
subs = pysubs2.load_from_whisper(results)
srt_path = transcription_root + new_filename
subs.save(srt_path)
print(f"SRT file saved at: {srt_path}")

# Optionally generate plain text file
if Generate_Plain_Document.lower() == "yes":
    txt_path = transcription_root + new_filename2
    with open(txt_path, "w") as txt_file:
        for segment in results:
            txt_file.write(f"{segment['text']}\n")
    print("Generated both SRT and TXT files.")

else:
    print("Generated only SRT file.")

print("Transcription complete!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

Whisper model loaded.
Detected language 'zh' with probability 0.996094
Starting transcription with verbose output...
[0.00s -> 14.00s] 接下來就來帶大家看一下GPT商店
[14.00s -> 18.00s] 然後教大家怎麼找到機器人的GPT
[18.00s -> 22.00s] 基本上在這邊左上角會有個Explore GPT
[22.00s -> 24.00s] 或者是如果你有中文的話
[24.00s -> 30.00s] 會看到這邊是一個探索GPTS
[30.00s -> 32.00s] 然後你就會看到
[32.00s -> 36.00s] 它上面就有很多很多人家寫好的GPT
[36.00s -> 38.00s] 寫作相關的GPT
[38.00s -> 40.00s] 會有這個生產力
[40.00s -> 43.00s] 就是幫你生產什麼Powerpoint的生產海報
[43.00s -> 45.00s] 生產PDF的這種GPT
[45.00s -> 49.00s] 或者是研究跟分析資料的GPT
[49.00s -> 50.00s] 它有很多很多
[50.00s -> 53.00s] 大家有興趣可以自己研究看看
[53.00s -> 55.00s] 你就在這邊打上機器人
[55.00s -> 57.00s] 你就把它點進來
[57.00s -> 60.00s] 然後你就會看到它的一些資訊
[60.00s -> 61.00s] 這我們等一下會講
[61.00s -> 65.00s] 阿Chet開始跟機器人聊天
[65.00s -> 69.00s] 這樣你就可以找到機器人GPT了
[69.00s -> 71.00s] 基本上先講大條件
[71.00s -> 73.00s] 你要製作GPT你要付錢
[73.00s -> 78.00s] 你要付GPT它的Plus訂閱費
[78.00s -> 80.00s] 免費仔是不能自己做的
[80.00s -> 82.00s] 但是免費仔可以用
[82.00s -> 84.00s] 付費仔做出來的GPT
[84.00s -> 86.00s] 如果你要建立自己的GPT的話
[86.00s -> 90.00

# 壓縮檔案並上傳
如果雲端硬碟被停用了

In [ ]:
if disable_google_drive.lower() == 'yes':
    # Creating the ZIP file
    filenametime = datetime.now().strftime("%m-%d-%Y-%H-%M-%S")
    archived = shutil.make_archive(filenametime, 'zip', '/content/result')
    if os.path.exists(archived):
      print(archived)
    if shutdown_after_complete.lower() == "yes":
      print(f"Turning off Auto Shutdown. Due to file download doesn't support when the runtime is shutdown.")
      shutdown_after_complete = "no"
    files.download(archived)
    print("File downloaded successfully.")

# 自動關閉執行階段

In [ ]:
import time
end_time = datetime.now()
print('\u751f\u7522\u7e3d\u6642\u9577: {}'.format(end_time - start_time))
time.sleep (2)

if shutdown_after_complete.lower() == "yes":
    print("Shutting Down the Runtime Because Shutdown After Complete is on")
    from google.colab import runtime
    runtime.unassign()


生產總時長: 0:01:36.578171
Shutting Down the Runtime Because Shutdown After Complete is on
